In [1]:
# masked, t_bound 0.5 and t_ext 0.1

import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
import shutil
from skimage import measure
workhorse = True

if workhorse:
    origin = 'Aldhani/eoagritwin/'
else:
    origin = ''

# transform unique-pairs array to dict
def unique_dict(unique_pairs_array):
    valid_dict = {}

    for key, value in unique_pairs_array:
        if key in valid_dict:
            valid_dict[key].append(value)
        else:
            valid_dict[key] = [value]

    return valid_dict

def make2000000(x):
    s = str(x)
    if len(s) == 7:
        return int('2' + s[1:] )
    else:
        return int('2' + s[2:] )

In [ ]:
# get tile files for prediction
path = f'/data/{origin}fields/Auxiliary/grid_search/Brandenburg/256_20_chips_masked_with_and_preds_are_GSA-DE_BRB-2019_cropMask_lines_touch_true_lines_touch_true_linecrop_prediction_extent/intermediates/'
files = getFilelist(path, '.tif')
files_sub = [file for file in files if '0.1_0.5' in file if '_pred_' in file]

# make a physical copy of these files
outpath = path.split('256_20')[0] + 'quick_n_dirty/'
if len(getFilelist(outpath, '.tif')) != 0:
    for file in getFilelist(outpath, '.tif'):
        os.remove(file)
for file in files_sub:
    shutil.copy2(file, outpath + file.split('intermediates/')[-1])

files_sub = getFilelist(outpath, '.tif')

In [3]:
# loop over files to get row and col cuts
rows, cols = [], []
for file in files_sub:
    rows.append(int(file.split('_instance_pred_')[-1].split('_')[0]))
    cols.append(int(file.split('_instance_pred_')[-1].split('_')[-1].split('.')[0]))
rows.sort()
cols.sort()
rows = list(set(rows))
cols = list(set(cols))
rows.sort()
cols.sort()

In [4]:
##### iterate over possible row/col connections and search for fields in neighbouring tiles
# start at top-left-corner
for row in rows:
    for col in cols:

        # row = rows[0]
        # col = cols[5]
        tile1 = [sub for sub in files_sub if f'_{row}_{col}.tif' in sub]
        if len(tile1) == 0:
            continue
        else: 
            ### check for neighbouring tiles and store result in list
            #print(f'_{row}_{col}.tif')
            # upper  (row -1)
            if rows.index(row)!=0:
                upper = [sub for sub in files_sub if f'_{rows[rows.index(row)-1]}_{col}.tif' in sub]
            else:
                upper = []

            # upper right (row -1 and col +1)
            if rows.index(row)!=0 and cols.index(col) < len(cols) - 1:
                upper_right = [sub for sub in files_sub if f'_{rows[rows.index(row)-1]}_{cols[cols.index(col)+1]}.tif' in sub]
            else:
                upper_right = []

            # right (col +1)
            if cols.index(col) < len(cols) - 1:
                right = [sub for sub in files_sub if f'_{row}_{cols[cols.index(col)+1]}.tif' in sub]
            else:
                right = []

            # lower right (row +1 and col +1)
            if rows.index(row) < len(rows) - 1 and cols.index(col) < len(cols) - 1:
                lower_right = [sub for sub in files_sub if f'_{rows[rows.index(row)+1]}_{cols[cols.index(col)+1]}.tif' in sub]
            else:
                lower_right = []

            # bottom (row +1)
            if rows.index(row) < len(rows) - 1:
                bottom = [sub for sub in files_sub if f'_{rows[rows.index(row)+1]}_{col}.tif' in sub]
            else:
                bottom = []

            if any(len(lst) > 0 for lst in [upper, upper_right, right, lower_right, bottom]):
                # load starting tile
                ds = gdal.Open(tile1[0])
                dat = ds.GetRasterBand(1).ReadAsArray()
                
                ### check the other tiles
                # upper
                if len(upper) == 1:
                    ds = gdal.Open(upper[0])
                    neighbour = ds.GetRasterBand(1).ReadAsArray()
                    
                    unique_pairs = np.unique(np.stack((dat[0,:], neighbour[-1,:]), axis=1), axis=0)
                    valid_pairs = unique_pairs[(unique_pairs != 0).all(axis=1)]
                    valid_dict = unique_dict(valid_pairs)
                    for v, p_list in valid_dict.items():
                        dat[dat == v] = 2000000 + v
                        for p in p_list:
                            neighbour[neighbour == p] = 2000000 + v
                    # export manipulated tile (will overwrite)
                    makeTif_np_to_matching_tif(neighbour, upper[0], upper[0], noData=0)

                # upper right
                if len(upper_right) == 1:
                    ds = gdal.Open(upper_right[0])
                    neighbour = ds.GetRasterBand(1).ReadAsArray()
                    
                    unique_pairs = np.unique(np.stack((dat[0,-1], neighbour[-1,0])), axis=0)
                    valid_pairs = unique_pairs[(unique_pairs != 0).all()]
                    if len(valid_pairs) > 1:
                        for v, p in valid_pairs:
                            dat[dat == v] = 2000000 + v
                            neighbour[neighbour == p] = 2000000 + v
                    # export manipulated tile (will overwrite)
                    makeTif_np_to_matching_tif(neighbour, upper_right[0], upper_right[0], noData=0)

                # right
                if len(right) == 1:
                    ds = gdal.Open(right[0])
                    neighbour = ds.GetRasterBand(1).ReadAsArray()
                    
                    unique_pairs = np.unique(np.stack((dat[:,-1], neighbour[:,0]), axis=1), axis=0)
                    valid_pairs = unique_pairs[(unique_pairs != 0).all(axis=1)]
                    valid_dict = unique_dict(valid_pairs)
                    for v, p_list in valid_dict.items():
                        dat[dat == v] = 2000000 + v
                        for p in p_list:
                            neighbour[neighbour == p] = 2000000 + v
                    # export manipulated tile (will overwrite)
                    makeTif_np_to_matching_tif(neighbour, right[0], right[0], noData=0)

                # lower right
                if len(lower_right) == 1:
                    ds = gdal.Open(lower_right[0])
                    neighbour = ds.GetRasterBand(1).ReadAsArray()
                    
                    unique_pairs = np.unique(np.stack((dat[-1,-1], neighbour[0,0])), axis=0)
                    valid_pairs = unique_pairs[(unique_pairs != 0).all()]
                    if len(valid_pairs) > 1:
                        for v, p in valid_pairs:
                            dat[dat == v] = 2000000 + v
                            neighbour[neighbour == p] = 2000000 + v
                    # export manipulated tile (will overwrite)
                    makeTif_np_to_matching_tif(neighbour, lower_right[0], lower_right[0], noData=0)

                # bottom
                if len(bottom) == 1:
                    ds = gdal.Open(bottom[0])
                    neighbour = ds.GetRasterBand(1).ReadAsArray()

                    unique_pairs = np.unique(np.stack((dat[-1,:], neighbour[0,:]), axis=1), axis=0)
                    valid_pairs = unique_pairs[(unique_pairs != 0).all(axis=1)]
                    valid_dict = unique_dict(valid_pairs)
                    for v, p_list in valid_dict.items():
                        dat[dat == v] = 2000000 + v
                        for p in p_list:
                            neighbour[neighbour == p] = 2000000 + v
                    # export manipulated tile (will overwrite)
                    makeTif_np_to_matching_tif(neighbour, bottom[0], bottom[0], noData=0)

                makeTif_np_to_matching_tif(dat, tile1[0], tile1[0], noData=0)
            else:
                continue

 

          

/home/potzschf/mambaforge/envs/workhorse/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [5]:
# loop over every tile again and clean up mess (e.g. 20000059, 40000059, 60000059)
vec_func = np.vectorize(make2000000,otypes=[int])

for file in files_sub:
    ds = gdal.Open(file)
    arr = ds.GetRasterBand(1).ReadAsArray()
    mask = arr > 3000000
    arr[mask] = vec_func(arr[mask])
    makeTif_np_to_matching_tif(arr, file, file, noData=0)

In [ ]:
# make a vrt and redo rasterIDs
gdal.BuildVRT(f'{outpath}quick_n_dirty.vrt', files_sub)
vrt = None

In [3]:
ds = gdal.Open(f'{outpath}quick_n_dirty.vrt')
block = ds.GetRasterBand(1).ReadAsArray()
relabelled = measure.label(block, background=0, connectivity=1)

/data/Aldhani/users/potzschf/conda/envs/workhorse/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [ ]:
#### polygonize
# create an in-memory raster-band with geoinfo of the relabelled array
rows, cols = relabelled.shape
driver = gdal.GetDriverByName('MEM')  # In-memory raster
raster_ds = driver.Create('', cols, rows, 1, gdal.GDT_Int32)
raster_ds.GetRasterBand(1).WriteArray(relabelled)
raster_ds.SetGeoTransform(ds.GetGeoTransform())
raster_ds.SetProjection(ds.GetProjection())
src_band = raster_ds.GetRasterBand(1)

# create a mask for the background (otherwise it would get a polygon as well)
mask_array = (relabelled != 0).astype(np.uint8)
mask_ds = driver.Create('', cols, rows, 1, gdal.GDT_Int32)
mask_ds.GetRasterBand(1).WriteArray(mask_array)
mask_band = mask_ds.GetRasterBand(1)

# create output for shp
driver = ogr.GetDriverByName('ESRI Shapefile')  # or 'GeoJSON', 'GPKG', etc.
out_ds = driver.CreateDataSource(f'{outpath}Fields_polygons.shp')  # Output vector file
out_layer = out_ds.CreateLayer('polygons', getSpatRefRas(ds), geom_type=ogr.wkbPolygon)
field_defn = ogr.FieldDefn('FieldID', ogr.OFTInteger)
out_layer.CreateField(field_defn)

# polygonize
gdal.Polygonize(src_band, mask_band, out_layer, 0, [], callback=None)
del out_ds

In [19]:
import geopandas as gpd

gdf = gpd.read_file(f'{outpath}Fields_polygons.shp')
check = gdf['FieldID']
ids, counts = np.unique(check, return_counts=True)
print(np.sum(counts > 1))


0


In [ ]:
# JUST TO EXPORT THE RASTER !!!!!

tif_driver = gdal.GetDriverByName('GTiff')

#### polygonize
# create an in-memory raster-band with geoinfo of the relabelled array
rows, cols = relabelled.shape
driver = gdal.GetDriverByName('MEM')  # In-memory raster
raster_ds = driver.Create('', cols, rows, 1, gdal.GDT_Int32)
raster_ds.GetRasterBand(1).WriteArray(relabelled)
raster_ds.SetGeoTransform(ds.GetGeoTransform())
raster_ds.SetProjection(ds.GetProjection())
src_band = raster_ds.GetRasterBand(1)

# create a mask for the background (otherwise it would get a polygon as well)
mask_array = (relabelled != 0).astype(np.uint8)
mask_ds = driver.Create('', cols, rows, 1, gdal.GDT_Int32)
mask_ds.GetRasterBand(1).WriteArray(mask_array)
mask_band = mask_ds.GetRasterBand(1).ReadAsArray()

tif_ds = tif_driver.CreateCopy(f'{outpath}BRA_MASK.tif', raster_ds, strict=0)
tif_ds.GetRasterBand(1).SetNoDataValue(0)
tif_ds.FlushCache()
tif_ds = None

